In [1]:
from pathlib import Path

In [36]:
import pandas as pd
import geopandas as gpd
import rasterstats as rs

In [5]:
base_path = Path("/home/jovyan/data")

pre_ndvi_path = base_path / "ndvi" / "output" / "yearly" / "avhrr_ndvi_v5_2001.tif"
post_ndvi_path = base_path / "ndvi" / "output" / "yearly" / "avhrr_ndvi_v5_2011.tif"

pre_pop_path = base_path / "population" / "ppp_2001_1km_Aggregated.tif"
post_pop_path = base_path / "population" / "ppp_2011_1km_Aggregated.tif"

gcdf_path = base_path / "gcdf" / "all_combined_global.gpkg"

In [6]:
gcdf_gdf = gpd.read_file(gcdf_path, driver="GPKG")

/opt/conda/lib/python3.11/site-packages/pyogrio/raw.py:196: RuntimeWarning: driver GPKG does not support open option DRIVER
  return ogr_read(


In [22]:
mining_project_id_list = [256, 684, 694, 30068, 30100, 30396, 30950, 33786, 34115, 34207, 34478, 35306, 36064, 36134, 36135, 37231, 37480, 37541, 37914, 37940, 38159, 38293, 39557, 39943, 39997, 40198, 40267, 40326, 40479, 41175, 41176, 41600, 41941, 42515, 43054, 47065, 49813, 52175, 52483, 53453, 53755, 54315, 54478, 54489, 54832, 55410, 55424, 55426, 55427, 56098, 56099, 56386, 57211, 57683, 58675, 58676, 59968, 60236, 60249, 61056, 61142, 61252, 61904, 62117, 62119, 62273, 62275, 62557, 63266, 63663, 63670, 63778, 64096, 64488, 64491, 64520, 64583, 64615, 64616, 64653, 64763, 65745, 65848, 66207, 66216, 66369, 66382, 66397, 66466, 66587, 66590, 66670, 66678, 66689, 67074, 67085, 67103, 67801, 67852, 68929, 69238, 69239, 69488, 70000, 70174, 70220, 70235, 70301, 70319, 70407, 70430, 71391, 72514, 72776, 73061, 73145, 73204, 73371, 85267, 85371, 85372, 85817, 85885, 86577, 87033, 87514, 88007, 88341, 88390, 88608, 88843, 89485, 89486, 89494, 89495, 89496, 89497, 89498, 89500, 89502, 89508, 89509, 89512, 91509, 91511, 91801, 91886, 91890, 91895, 91897, 91899, 91902, 91923, 91926, 91968, 91977, 91979, 91980, 92120, 92353, 92367, 92461, 92497, 92518]

start_year = 2001
end_year = 2011

In [23]:
mining_gdf = gcdf_gdf.loc[gcdf_gdf.id.isin(mining_project_id_list) & (gcdf_gdf["Commitment.Year"] > start_year) & (gcdf_gdf["Completion.Year"] < end_year)].copy()

In [40]:
pre_ndvi_extract = rs.zonal_stats(mining_gdf.geometry, pre_ndvi_path, stats="mean", geojson_out=True, all_touched=True)
post_ndvi_extract = rs.zonal_stats(mining_gdf.geometry, post_ndvi_path, stats="mean", geojson_out=True, all_touched=True)

pre_ndvi_vals = [(i["id"], i["properties"]["mean"]) for i in pre_ndvi_extract]
post_ndvi_vals = [(i["id"], i["properties"]["mean"]) for i in post_ndvi_extract]

In [41]:
pre_pop_extract = rs.zonal_stats(mining_gdf.geometry, pre_pop_path, stats="mean", geojson_out=True, all_touched=True)
post_pop_extract = rs.zonal_stats(mining_gdf.geometry, post_pop_path, stats="mean", geojson_out=True, all_touched=True)

pre_pop_vals = [(i["id"], i["properties"]["mean"]) for i in pre_pop_extract]
post_pop_vals = [(i["id"], i["properties"]["mean"]) for i in post_pop_extract]

In [79]:
df_list = [
    pd.DataFrame(pre_ndvi_vals, columns=["id", "ndvi_2001"]),
    pd.DataFrame(post_ndvi_vals, columns=["id", "ndvi_2011"]),
    pd.DataFrame(pre_pop_vals, columns=["id", "pop_2001"]),
    pd.DataFrame(post_pop_vals, columns=["id", "pop_2011"]),
]

df = pd.concat(df_list, axis=1)
df = df.loc[:,~df.columns.duplicated()].copy()
df.id = df.id.astype(int)
df = df.set_index("id")


In [84]:

gcdf_fields = ["id", "Recipient", "Recipient.ISO-3", "Title", "Status", "Amount.(Constant.USD.2021)"]
dfx = df.merge(mining_gdf[gcdf_fields], left_index=True, right_index=True)
dfx.to_csv("/home/jovyan/data/extract.csv", index=False)